# Import Libraries

In [764]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
from math import erf, sqrt, log, exp

In [765]:
# 1. BRFSS 2015 (from Kaggle)
df_brfss = pd.read_csv("diabetes_binary_5050split_health_indicators_BRFSS2015.csv")

# standardising column names 
df_brfss.columns = df_brfss.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-','_')

In [766]:
df_brfss.head()

,diabetes_binary,highbp,highchol,cholcheck,bmi,smoker,stroke,heartdiseaseorattack,physactivity,fruits,...,anyhealthcare,nodocbccost,genhlth,menthlth,physhlth,diffwalk,sex,age,education,income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [767]:
df_brfss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   diabetes_binary       70692 non-null  float64
 1   highbp                70692 non-null  float64
 2   highchol              70692 non-null  float64
 3   cholcheck             70692 non-null  float64
 4   bmi                   70692 non-null  float64
 5   smoker                70692 non-null  float64
 6   stroke                70692 non-null  float64
 7   heartdiseaseorattack  70692 non-null  float64
 8   physactivity          70692 non-null  float64
 9   fruits                70692 non-null  float64
 10  veggies               70692 non-null  float64
 11  hvyalcoholconsump     70692 non-null  float64
 12  anyhealthcare         70692 non-null  float64
 13  nodocbccost           70692 non-null  float64
 14  genhlth               70692 non-null  float64
 15  menthlth           

In [768]:
# distribution of classes in dataset
print(df_brfss['diabetes_binary'].value_counts())
print(df_brfss['diabetes_binary'].value_counts(normalize=True))

diabetes_binary
0.0    35346
1.0    35346
Name: count, dtype: int64
diabetes_binary
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64


# Hypothesis 1: Lifestyle Habits and Diabetes

### Individual Lifestyle Factors vs. Diabetes 
*Compare diabetes prevalence for people with vs without specific lifestyle risk factors, add uncertainty (Wilson 95% CI), and show a clean grouped-bar chart.*

In [769]:
FACTOR_ORDER = [
    ("smoker", "Smoking"),
    ("physactivity", "No Physical Activity"), 
    ("fruits", "Low Fruit Intake"),            
    ("veggies", "Low Veggie Intake"),         
]
RISK_VALUE = {"smoker": 1, "physactivity": 0, "fruits": 0, "veggies": 0}

def wilson(success, n, z=1.96):
    if n == 0: return (np.nan, np.nan)
    p = success / n
    denom = 1 + z**2/n
    center = (p + z**2/(2*n)) / denom
    half = (z*np.sqrt((p*(1-p) + z**2/(4*n)) / n)) / denom
    return center - half, center + half

def row_stats(df, col, risk_value):
    # with risk
    a = df[df[col] == risk_value]['diabetes_binary']
    n1, c1 = a.count(), a.sum()
    p1 = c1 / n1 if n1 else np.nan
    lo1, hi1 = wilson(c1, n1)
    # without risk
    b = df[df[col] != risk_value]['diabetes_binary']
    n0, c0 = b.count(), b.sum()
    p0 = c0 / n0 if n0 else np.nan
    lo0, hi0 = wilson(c0, n0)
    return p1, lo1, hi1, n1, int(c1), p0, lo0, hi0, n0, int(c0)

rows = []
for col, label in FACTOR_ORDER:
    p1, lo1, hi1, n1, c1, p0, lo0, hi0, n0, c0 = row_stats(df_brfss, col, RISK_VALUE[col])
    rows.append({
        "Factor": label,
        "with_prev": p1, "with_lo": lo1, "with_hi": hi1, "with_n": n1, "with_cases": c1,
        "without_prev": p0, "without_lo": lo0, "without_hi": hi0, "without_n": n0, "without_cases": c0
    })
tbl = pd.DataFrame(rows)


ymax = float(pd.concat([tbl["with_prev"], tbl["without_prev"]]).max()) * 1.25
ymax = max(0.05, ymax)

fig = go.Figure()

# soft alternating horizontal bands (every 10%)
for i, y0 in enumerate(np.arange(0, ymax, 0.10)):
    if i % 2 == 0:
        fig.add_hrect(y0=y0, y1=min(y0+0.10, ymax),
                      fillcolor="#F2F4F7", line_width=0, layer="below")

hover = "Factor: %{x}<br>%{fullData.name}: %{y:.1%}<br>n=%{customdata[0]}  cases=%{customdata[1]}<extra></extra>"

fig.add_bar(
    name="With Risk", legendgroup="With Risk",
    x=tbl["Factor"], y=tbl["with_prev"],
    error_y=dict(type="data", array=tbl["with_hi"]-tbl["with_prev"],
                 arrayminus=tbl["with_prev"]-tbl["with_lo"], thickness=1),
    text=tbl["with_prev"].map(lambda v:f"{v:.1%}"), textposition="outside",
    marker=dict(color="#A64A47", line=dict(color="rgba(0,0,0,0.15)", width=1)),
    customdata=np.stack([tbl["with_n"], tbl["with_cases"]], axis=-1),
    hovertemplate=hover,
)

fig.add_bar(
    name="Without Risk", legendgroup="Without Risk",
    x=tbl["Factor"], y=tbl["without_prev"],
    error_y=dict(type="data", array=tbl["without_hi"]-tbl["without_prev"],
                 arrayminus=tbl["without_prev"]-tbl["without_lo"], thickness=1),
    text=tbl["without_prev"].map(lambda v:f"{v:.1%}"), textposition="outside",
    marker=dict(color="#E8C6AE", line=dict(color="rgba(0,0,0,0.12)", width=1)),
    customdata=np.stack([tbl["without_n"], tbl["without_cases"]], axis=-1),
    hovertemplate=hover,
)

fig.update_traces(cliponaxis=False, textfont_size=11)

fig.update_layout(
    title=dict(text="Individual Lifestyle Factors<br><sup>Diabetes rate with vs without risk</sup>",
               x=0.02, xanchor="left"),
    barmode="group", bargap=0.28, bargroupgap=0.14,
    xaxis=dict(title="", showgrid=False, tickangle=0),
    yaxis=dict(title="Diabetes Rate (%)", tickformat=".0%", dtick=0.10,
               range=[0, ymax], showgrid=False),
    legend=dict(orientation="h", x=0.5, xanchor="center", y=-0.18),
    template="simple_white",
    paper_bgcolor="white", plot_bgcolor="white",
    margin=dict(l=70, r=30, t=80, b=90)
)

fig.show()



### Diabetes prevalence increases with multiple lifestyle risk factors
*Count lifestyle risks per person (smoking, inactivity, low fruit/veg, heavy alcohol), compute diabetes prevalence for 0,1,2,3,4+ factors, and plot a clean bar chart (with soft bands) to show the increase in prevalence as risks accumulate.*

In [770]:
# Build a simple risk count (smoker=1, physactivity=0, fruits=0, veggies=0, hvyalcoholconsump=1 if present)
rb = pd.DataFrame(index=df.index)
if "smoker" in df:               rb["smoke"]    = (df["smoker"]==1).astype(int)
if "physactivity" in df:         rb["inactive"] = (df["physactivity"]==0).astype(int)
if "fruits" in df:               rb["lowfruit"] = (df["fruits"]==0).astype(int)
if "veggies" in df:              rb["lowveg"]   = (df["veggies"]==0).astype(int)
if "hvyalcoholconsump" in df:    rb["heavyalc"] = (df["hvyalcoholconsump"]==1).astype(int)

df["risk_behaviors"] = rb.sum(axis=1).fillna(0).astype(int)

# Prevalence by exact risk count
prev_rb = (df.groupby("risk_behaviors")["diabetes_binary"]
             .mean()
             .rename("prevalence")
             .reset_index())

# Collapse to display groups (0,1,2,3,4+)
prev_rb["group"] = prev_rb["risk_behaviors"].apply(
    lambda k: "4+ factors" if k>=4 else f"{k} factor{'s' if k!=1 else ''}"
)

# Average prevalence within each display group and order nicely
prev_rb = (prev_rb.groupby("group", as_index=False)["prevalence"].mean())
order = ["0 factors","1 factor","2 factors","3 factors","4+ factors"]
prev_rb["group"] = pd.Categorical(prev_rb["group"], categories=order, ordered=True)
prev_rb = prev_rb.sort_values("group")

# Plot 
ymax = max(0.05, float(prev_rb["prevalence"].max()) * 1.25)
step = 0.10

# bars
fig = go.Figure(go.Bar(
    x=prev_rb["group"],
    y=prev_rb["prevalence"],
    text=prev_rb["prevalence"].map(lambda v: f"{v:.1%}"),
    textposition="outside",
    marker=dict(color="#a64a47", line=dict(width=1, color="rgba(0,0,0,0.15)")),
    hovertemplate="<b>%{x}</b><br>Diabetes rate: %{y:.1%}<extra></extra>",
    cliponaxis=False
))

# soft alternating background bands (one-liner)
fig.update_layout(shapes=[
    dict(type="rect", xref="paper", x0=0, x1=1,
         y0=y, y1=min(y+step, ymax), layer="below",
         line=dict(width=0), fillcolor="#F2F4F7")
    for i, y in enumerate(np.arange(0, ymax, step)) if i % 2 == 0
])

fig.update_layout(
    title=dict(text="Diabetes prevalence increases with multiple lifestyle risk factors", x=0.02),
    template="simple_white",
    bargap=0.35, showlegend=False,
    margin=dict(l=60, r=20, t=60, b=60),
    xaxis=dict(title="Number of factors", categoryorder="array",
               categoryarray=["0 factors","1 factor","2 factors","3 factors","4+ factors"],
               showgrid=False),
    yaxis=dict(title="Diabetes Rate (%)", tickformat=".0%", dtick=0.10, range=[0, ymax], showgrid=False),
    uniformtext_minsize=10, uniformtext_mode="hide",
    paper_bgcolor="white", plot_bgcolor="white"
)

fig.show()


### Relative Risk Radar
*Compute the % change in diabetes prevalence for each factor and visualize it on a radar chart.*

In [771]:
# Radar: % increase = (with/without - 1) * 100 
import math
import plotly.graph_objects as go

df_bars["% Increase"] = (df_bars["With Risk"] / df_bars["Without Risk"] - 1) * 100

# Categories + values
cats = df_bars["Factor"].tolist()
vals = df_bars["% Increase"].tolist()
cats_closed = cats + cats[:1]
vals_closed = vals + vals[:1]

# Axis bounds with padding (handles negatives)
vmax = max(vals) if vals else 0
vmin = min(vals) if vals else 0
pad_hi = 0.10 * (abs(vmax) if vmax else 1)
pad_lo = 0.10 * (abs(vmin) if vmin else 1)
rmax = max(10, math.ceil((vmax + pad_hi) / 10.0) * 10.0)
rmin = math.floor((min(0, vmin - pad_lo)) / 10.0) * 10.0

hover = [
    (f"<b>{row.Factor}</b><br>"
     f"With risk: {row['With Risk']:.1%}<br>"
     f"Without risk: {row['Without Risk']:.1%}<br>"
     f"Δ risk: {row['% Increase']:+.1f}%")
    for _, row in df_bars.iterrows()
]
hover_closed = hover + hover[:1]

line_color = "#a64a47"
fill_color = "rgba(166, 74, 71, 0.22)"

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    theta=cats_closed, r=vals_closed,
    mode="lines",
    line=dict(color=line_color, width=3, shape="spline"),
    fill="toself", fillcolor=fill_color,
    hoveron="points+fills",            
    hoverinfo="text", hovertext=hover_closed,
    showlegend=False
))

fig.add_trace(go.Scatterpolar(
    theta=cats, r=vals,
    mode="markers+text",
    marker=dict(size=8, line=dict(width=1, color="rgba(0,0,0,0.2)"), color=line_color),
    text=[f"{v:+.0f}%" for v in vals],
    textposition="top center",
    hoverinfo="text",                 
    hovertext=hover,                   
    showlegend=False
))


fig.update_layout(
    title=dict(
        text="Relative Risk Comparison<br><sup>% change in diabetes rate by factor</sup>",
        x=0.03, xanchor="left", font=dict(size=22)
    ),
    template="simple_white",
    paper_bgcolor="white",
    polar=dict(
        bgcolor="white",
        angularaxis=dict(
            rotation=90, direction="clockwise",
            gridcolor="#e9eef3", gridwidth=1, tickfont=dict(size=12)
        ),
        radialaxis=dict(
            visible=True, range=[rmin, rmax],
            tick0=0, dtick=10, ticksuffix="%",
            gridcolor="#e9eef3", gridwidth=1, tickfont=dict(size=12)
        )
    ),
    margin=dict(l=40, r=40, t=80, b=40)
)

fig.show()


### Effect of Lifestyle Factors on Diabetes Rate (Δ% vs No-Risk, 95% CI)

*This graph visualizes how specific lifestyle risk factors — smoking, lack of physical activity, low fruit intake, and low vegetable intake — affect the likelihood of having diabetes compared to people without those risks.*


In [772]:
FACTORS = [("smoker","Smoking",1), ("physactivity","No Physical Activity",0),
           ("fruits","Low Fruit Intake",0), ("veggies","Low Veggie Intake",0)]

rows=[]
y = df["diabetes_binary"].astype(int)
for col,label,risk in FACTORS:
    if col not in df: continue
    m = df[col].eq(risk)
    a, n1 = int(y[m].sum()), int(m.sum())           # with risk: cases, total
    c, n0 = int(y[~m].sum()), int((~m).sum())       # without risk: cases, total
    p1, p0 = a/n1, c/n0
    # two-proportion z p-value (pooled)
    p = (a+c)/(n1+n0); se = sqrt(p*(1-p)*(1/n1+1/n0)); z = 0 if se==0 else (p1-p0)/se
    pval = 2*(1-0.5*(1+erf(abs(z)/sqrt(2))))
    # log-RR 95% CI -> % change
    rr = p1/p0 if p0>0 else np.nan
    se_rr = sqrt((1/max(a,1))-(1/max(n1,1))+(1/max(c,1))-(1/max(n0,1)))
    lo, hi = (np.nan, np.nan) if not rr or rr<=0 else (exp(log(rr)-1.96*se_rr), exp(log(rr)+1.96*se_rr))
    rows.append(dict(Factor=label, pct=(rr-1)*100, lo=(lo-1)*100, hi=(hi-1)*100,
                     with_=f"{a}/{n1}", without_=f"{c}/{n0}", pval=pval))

out = pd.DataFrame(rows).dropna().sort_values("pct")
out["err+"] = out["hi"]-out["pct"]; out["err-"] = out["pct"]-out["lo"]
colors = np.where(out["pval"]<0.05, "#A64A47", "#9AA4B2")

fig = go.Figure()
fig.add_vline(x=0, line_dash="dash", line_color="black")
fig.add_scatter(
    x=out["pct"], y=out["Factor"], mode="markers",
    marker=dict(size=10, color=colors),
    error_x=dict(type="data", array=out["err+"], arrayminus=out["err-"]),
    customdata=np.c_[out["lo"],out["hi"],out["with_"],out["without_"],out["pval"]],
    hovertemplate=("<b>%{y}</b><br>Δ rate: %{x:.1f}% "
                   "(95% CI %{customdata[0]:.1f}–%{customdata[1]:.1f}%)<br>"
                   "With: %{customdata[2]} · Without: %{customdata[3]}<br>"
                   "p=%{customdata[4]:.3g}<extra></extra>")
)
fig.update_layout(
    title="Effect of Lifestyle Factors on Diabetes Rate (Δ% vs no-risk, 95% CI)",
    template="simple_white",
    xaxis_title="Percent change (%)", yaxis_title="Factors",
    margin=dict(l=80,r=40,t=70,b=40)
)
fig.show()


# Hypothesis 2 — Education & Diabetes Prevention

### Education & Diabetes  
*Group respondents by education level and diabetes status, then plot a stacked bar chart of counts to show how the mix of “Diabetes” vs “No Diabetes” changes across education levels.*

In [773]:
edu_diab = (
    df_brfss.groupby(["education","diabetes_binary"])
    .size().reset_index(name="count")
)
edu_diab["diabetes_binary"] = edu_diab["diabetes_binary"].map({0:"No Diabetes",1:"Diabetes"})

fig = px.bar(
    edu_diab, x="education", y="count", color="diabetes_binary",
    title="Distribution of Diabetes Status by Education Level",
    barmode="relative", text_auto=True, color_discrete_sequence=["#E8BAB9","#931A23"]
)
fig.update_layout(xaxis_title="Education level", yaxis_title="Proportion")


### Education & Health Behaviors
*Recode education into four levels, compute the % with Healthy Diet, Physical Activity, and Regular Checkups by education group, and visualize them as grouped bars to show healthier behaviors with higher education.*

In [774]:
edu_col = 'educa' if 'educa' in df.columns else 'education'
map4 = {1:'Less than HS', 2:'Less than HS', 3:'Less than HS',
        4:'HS Graduate', 5:'Some College', 6:'College Grad'}
order4 = ['Less than HS','HS Graduate','Some College','College Grad']

df = df.dropna(subset=[edu_col]).copy()
df['Education'] = pd.Categorical(df[edu_col].astype(int).map(map4), categories=order4, ordered=True)

# metrics
if {'fruits','veggies'}.issubset(df.columns):
    df['Healthy Diet'] = ((df['fruits']==1) & (df['veggies']==1)).astype(int)
elif 'fruits' in df:  df['Healthy Diet'] = df['fruits'].astype(int)
else:                 df['Healthy Diet'] = df['veggies'].astype(int)

df['Physical Activity'] = df['physactivity'].astype(int)
df['Regular Checkups']  = (df['cholcheck'] if 'cholcheck' in df else df['anyhealthcare']).astype(int)

# Aggregate to % by the 4 groups 
agg = (df.groupby('Education')[['Healthy Diet','Physical Activity','Regular Checkups']]
         .mean().mul(100).reset_index())

# long form for grouped bars
long = agg.melt(id_vars='Education', var_name='Metric', value_name='Prevalence')

# Plot 
colors = {'Healthy Diet':'#8C1D18','Physical Activity':'#C94B44','Regular Checkups':"#ECD9C6"}

fig = px.bar(
    long, x='Education', y='Prevalence', color='Metric',
    barmode='group', text='Prevalence',
    color_discrete_map=colors,
    title='Higher education predicts better health behaviors across multiple domains',
    labels={'Prevalence':'Prevalence (%)'},
    category_orders={'Education': order4, 'Metric': ['Healthy Diet','Physical Activity','Regular Checkups']},
    template='simple_white'
)

# soft horizontal bands (every 20%)
fig.update_layout(shapes=[
    dict(type="rect", xref="paper", x0=0, x1=1, y0=y, y1=min(y+20,100),
         layer="below", line=dict(width=0), fillcolor="#F2F4F7")
    for i, y in enumerate(np.arange(0, 100, 20)) if i % 2 == 0
])

fig.update_traces(
    texttemplate='%{text:.0f}%',
    textposition='outside',
    cliponaxis=False,
    hovertemplate='<b>%{x}</b><br>%{fullData.name}: %{y:.0f}%<extra></extra>',
    selector=dict(type='bar')
)


fig.update_layout(
    yaxis=dict(range=[0,100], dtick=20, title='Prevalence (%)', showgrid=False),
    xaxis=dict(title='Education Level', showgrid=False),
    legend=dict(orientation='h', x=0.5, xanchor='center', y=-0.22),
    bargap=0.28, bargroupgap=0.16,
    margin=dict(l=70, r=40, t=80, b=100),
    uniformtext_minsize=10, uniformtext_mode='hide',
    paper_bgcolor='white', plot_bgcolor='white'
)

fig.show()


/var/folders/3y/lq5dk83j0f97375lxy5txvt80000gn/T/ipykernel_61673/2310137586.py:19: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### Education & Diabetes Trend 
*Recode education into 4 levels, compute diabetes prevalence with 95% Wilson CIs, and plot a line with markers and a shaded CI ribbon.
The chart shows diabetes rates decline as education increases across the four groups.*

In [775]:
# Data prep 
df = df_brfss.dropna(subset=['diabetes_binary']).copy()
edu_col = 'educa' if 'educa' in df.columns else 'education'

# Map to 4 ordered buckets
map4   = {1:'Less than HS', 2:'Less than HS', 3:'Less than HS',
          4:'HS Graduate',  5:'Some College', 6:'College Grad'}
order4 = ['Less than HS','HS Graduate','Some College','College Grad']

df = df.dropna(subset=[edu_col]).copy()
df['edu4'] = pd.Categorical(df[edu_col].astype(int).map(map4),
                            categories=order4, ordered=True)

# Prevalence + 95% Wilson CI
def wilson_ci(k, n, z=1.96):
    if n == 0: return (np.nan, np.nan)
    p = k / n
    d = 1 + z**2/n
    c = (p + z**2/(2*n)) / d
    h = (z*np.sqrt((p*(1-p)+z**2/(4*n))/n)) / d
    return (c - h, c + h)

g = df.groupby('edu4')['diabetes_binary']
tab = pd.DataFrame({
    'Education': g.mean().index,
    'rate': (g.mean()*100).values,
    'n': g.size().values,
    'k': g.sum().values
})
cis = tab.apply(lambda r: pd.Series(wilson_ci(r['k'], r['n'])), axis=1)
tab['lo'] = cis[0]*100
tab['hi'] = cis[1]*100

# Plot
x = tab['Education']
y = tab['rate']
lo = tab['lo']; hi = tab['hi']

fig = go.Figure()

# CI ribbon
fig.add_traces([
    go.Scatter(x=x, y=hi, mode='lines', line=dict(width=0), showlegend=False,
               hoverinfo='skip'),
    go.Scatter(x=x, y=lo, mode='lines', line=dict(width=0), fill='tonexty',
               fillcolor='rgba(166,74,71,0.12)', showlegend=False, hoverinfo='skip')
])

# Line + markers
fig.add_trace(go.Scatter(
    x=x, y=y, mode='lines+markers',
    line=dict(color='#A64A47', width=3),
    marker=dict(size=10, color='white', line=dict(color='#A64A47', width=2)),
    text=[f"{v:.1f}%" for v in y],
    textposition='top center',
    hovertemplate="%{x}<br>Diabetes Rate: %{y:.1f}%<br>n=%{customdata}<extra></extra>",
    customdata=tab['n'],
    name='Diabetes rate'
))

fig.update_layout(
    title="Diabetes Rate Declines with Higher Education (4 groups)",
    template='simple_white',
    font=dict(family="Inter, Arial, sans-serif", size=16),
    margin=dict(l=70, r=30, t=80, b=70),
    xaxis=dict(title="Education Level", tickangle=0),
    yaxis=dict(title="Diabetes Rate (%)",
               tickformat=".0f",  
               range=[0, max(5, float(tab['hi'].max())*1.15)],
               gridcolor='rgba(0,0,0,0.08)', zeroline=False),
)

fig.show()


/var/folders/3y/lq5dk83j0f97375lxy5txvt80000gn/T/ipykernel_61673/611117434.py:23: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

